# Oil Price Prediction

## Constants

In [ ]:
BASE_PATH = 'BASE_PATH_OF_THE_PROJECT'

time: 722 µs (started: 2024-06-11 09:48:36 +00:00)


In [ ]:
CURRENT_PROJECT = 'ref18'

time: 552 µs (started: 2024-06-11 09:48:37 +00:00)


## Necessary codes

In [ ]:
from google.colab import drive

drive.mount('/MyDrive', force_remount=True)

Mounted at /MyDrive
time: 7.53 s (started: 2024-06-11 09:48:37 +00:00)


In [ ]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 7.75 s (started: 2024-06-11 09:48:45 +00:00)


## Helper Functions

In [ ]:
def flatten(a):
  isChanged = None
  b = []
  while True:
    isChanged = False
    for x in a:
      try:
        b = b + list(x)
        isChanged = True
      except:
        b.append(x)
    a = b
    b = []
    if not isChanged:
      break
  return a

time: 614 µs (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
def show_models_results(model, train_input, train_output, test_input, test_output, squeeze=True):
  def check_and_calc_squeeze(inp):
    if squeeze:
      return np.squeeze(inp)
    else:
      return inp

  print("train and test MAE: ",
        mean_absolute_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
        mean_absolute_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))
  print("train and test RMSE: ",
        mean_squared_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0), squared=False),
        mean_squared_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0), squared=False))
  print("train and test MSE: ",
        mean_squared_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
        mean_squared_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))
  print("train and test MAPE: ",
        mean_absolute_percentage_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
        mean_absolute_percentage_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))
  print("train and test R2: ",
        r2_score(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
      r2_score(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))


time: 19.1 ms (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
def show_models_results_with_labels(y_true, y_pred):
  print("MAE: ",
        mean_absolute_error(y_true, y_pred))
  print("RMSE: ",
        mean_squared_error(y_true, y_pred, squared=False))
  print("MSE: ",
        mean_squared_error(y_true, y_pred))
  print("MAPE: ",
        mean_absolute_percentage_error(y_true, y_pred))
  print("R2: ",
        r2_score(y_true, y_pred))


time: 829 µs (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
def write_models_and_cluster(models, cluster, model_name, trial_number):
  main_writing_path = BASE_PATH + 'models/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  cluster_path = main_writing_path + '/cluster'
  models_path = main_writing_path + "/models"

  os.makedirs(cluster_path)
  pickle.dump(cluster, open(cluster_path + '/cluster_file', 'wb'))

  for i, model in enumerate(models):
    current_model_path = models_path + "/" + str(i)
    os.makedirs(current_model_path)
    model.save(current_model_path)

time: 1.9 ms (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
def read_models_and_cluster(model_name, trial_number):
  main_reading_path = BASE_PATH + 'models/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  cluster_path = main_reading_path + '/cluster'
  models_path = main_reading_path + "/models"

  cluster = pickle.load(open(cluster_path + '/cluster_file', 'rb'))

  models = []

  for model in (os.listdir(models_path)):
    current_model_path = models_path + "/" + model
    models.append(tf.keras.models.load_model(current_model_path))

  return cluster, models

time: 1.01 ms (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
def is_model_trained(model_name, trial_number):
  main_reading_path = BASE_PATH + 'models/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  return os.path.exists(main_reading_path)

time: 675 µs (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
def write_model_results(model_name, trial_number, y_train_pred, y_test_pred):
  main_writing_path = BASE_PATH + 'results/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  os.makedirs(main_writing_path)
  pd.DataFrame({'predictions': y_train_pred.squeeze()}).to_csv(main_writing_path + "/train.csv", index=False)
  pd.DataFrame({'predictions': y_test_pred.squeeze()}).to_csv(main_writing_path + "/test.csv", index=False)

time: 862 µs (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
def read_model_results(model_name, trial_number):
  main_writing_path = BASE_PATH + 'results/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  train = pd.read_csv(main_writing_path + "/train.csv")
  test = pd.read_csv(main_writing_path + "/test.csv")
  return train, test

time: 774 µs (started: 2024-06-11 09:49:02 +00:00)


## Imports

In [ ]:
!pip install tslearn

time: 9.25 s (started: 2024-06-11 09:48:53 +00:00)


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from sklearn.cluster import KMeans
from tslearn.clustering import TimeSeriesKMeans
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import random
from keras.constraints import Constraint
from keras import backend as kerasBackend
import seaborn as sns
import pickle
import os

time: 905 µs (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
sns.set()

time: 1.61 ms (started: 2024-06-11 09:49:02 +00:00)


## Data Loading and Preparation

### Constants

In [ ]:
START_DATE = '1987-01-01'
END_DATE = '2019-12-31'
WTI_DATASET_PATH = BASE_PATH + 'data/processed-WTI-price-daily.csv'
BRENT_DATASET_PATH = BASE_PATH + 'data/processed-Brent-price-daily.csv'
TRAIN_PORTION = 0.7
VALIDATION_PORTION = 0
TEST_PORTION = 0.3
WINDOW_SIZE = 5

time: 950 µs (started: 2024-06-11 09:49:02 +00:00)


### Load data

In [ ]:
wti_whole = pd.read_csv(WTI_DATASET_PATH)
wti_whole

,Date,Price
0,1986-01-02,25.56
1,1986-01-03,26.00
2,1986-01-06,26.53
3,1986-01-07,25.85
4,1986-01-08,25.87
...,...,...
9579,2023-06-06,71.71
9580,2023-06-07,72.52
9581,2023-06-08,71.28
9582,2023-06-09,70.16


time: 44.7 ms (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
brent_whole = pd.read_csv(BRENT_DATASET_PATH)
brent_whole

,Date,Price
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63
...,...,...
9233,2023-06-06,76.22
9234,2023-06-07,76.85
9235,2023-06-08,75.88
9236,2023-06-09,74.74


time: 48.9 ms (started: 2024-06-11 09:49:02 +00:00)


### Get Wanted Period

In [ ]:
# wti_intersection = copy.deepcopy(wti_whole)
wti_intersection = wti_whole[wti_whole["Date"] >= START_DATE]
wti_intersection = wti_intersection[wti_intersection["Date"] <= END_DATE]
wti_intersection

,Date,Price
251,1987-01-02,18.13
252,1987-01-05,17.98
253,1987-01-06,18.21
254,1987-01-07,18.28
255,1987-01-08,18.63
...,...,...
8692,2019-12-24,61.17
8693,2019-12-26,61.72
8694,2019-12-27,61.76
8695,2019-12-30,61.66


time: 29.6 ms (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
# brent_intersection = copy.deepcopy(brent_whole)
brent_intersection = brent_whole[brent_whole["Date"] >= START_DATE]
brent_intersection = brent_intersection[brent_intersection["Date"] <= END_DATE]
brent_intersection

,Date,Price
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63
...,...,...
8346,2019-12-24,69.26
8347,2019-12-26,69.26
8348,2019-12-27,68.91
8349,2019-12-30,68.30


time: 43.7 ms (started: 2024-06-11 09:49:02 +00:00)


### Get mean and standard deviation for normalization

In [ ]:
wti_mean = np.mean(wti_intersection['Price'])
wti_std = np.std(wti_intersection['Price'])
print("WTI mean and std: ", wti_mean, wti_std)

WTI mean and std:  44.91450982713711 29.082753183526478
time: 8.53 ms (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
brent_mean = np.mean(brent_intersection['Price'])
brent_std = np.std(brent_intersection['Price'])
print("Brent mean and std: ", brent_mean, brent_std)

Brent mean and std:  46.38022991258532 32.623403721864776
time: 7 ms (started: 2024-06-11 09:49:02 +00:00)


### Divide into training and test

In [ ]:
number_of_datapoints_wti = len(wti_intersection)
number_of_datapoints_brent = len(brent_intersection)
number_of_datapoints_wti, number_of_datapoints_brent

(8446, 8351)

time: 4.29 ms (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
train_part_wti = math.floor(number_of_datapoints_wti * TRAIN_PORTION)
validation_part_wti = math.floor(number_of_datapoints_wti * VALIDATION_PORTION)
test_part_wti = math.floor(number_of_datapoints_wti * TEST_PORTION)

time: 778 µs (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
train_part_brent = math.floor(number_of_datapoints_brent * TRAIN_PORTION)
validation_part_brent = math.floor(number_of_datapoints_brent * VALIDATION_PORTION)
test_part_brent = math.floor(number_of_datapoints_brent * TEST_PORTION)

time: 777 µs (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
wti_train = wti_intersection.iloc[:train_part_wti].reset_index(drop=True)
wti_validation = wti_intersection.iloc[train_part_wti:train_part_wti+validation_part_wti].reset_index(drop=True)
wti_test = wti_intersection.iloc[train_part_wti+validation_part_wti:].reset_index(drop=True)
print('number of records of parts: ', [len(wti_train), len(wti_validation), len(wti_test)])

number of records of parts:  [5912, 0, 2534]
time: 9.57 ms (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
brent_train = brent_intersection.iloc[:train_part_brent].reset_index(drop=True)
brent_validation = brent_intersection.iloc[train_part_brent:train_part_brent+validation_part_brent].reset_index(drop=True)
brent_test = brent_intersection.iloc[train_part_brent+validation_part_brent:].reset_index(drop=True)
print('number of records of parts: ', [len(brent_train), len(brent_validation), len(brent_test)])

number of records of parts:  [5845, 0, 2506]
time: 9.05 ms (started: 2024-06-11 09:49:02 +00:00)


### Prepare inputs and targets

In [ ]:
import tensorflow as tf
import numpy as np
def get_input_and_output_tensor(df, feature_column, target_column, window_size):
  """
  inputs:
    df -> pandas dataframe
    feature_column -> column that is our feature
    target_column -> column that is our target
    window_size -> window size of inputs
  output:
    [inputs: (m ,window_size, 1), outputs: (m)]
  """
  inputs = np.zeros((len(df[feature_column]) - window_size, window_size, 1))
  outputs = np.zeros((len(df[feature_column]) - window_size))

  start = 0
  for i in range(window_size, len(df[feature_column])):
    inputs[start,:,0] = df[feature_column][start:i]
    outputs[start] = df[target_column][i]
    start = start + 1

  inputs = tf.convert_to_tensor(inputs)
  outputs = tf.convert_to_tensor(outputs)

  return [inputs, outputs]

time: 1.39 ms (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
wti_train_inputs, wti_train_outputs = get_input_and_output_tensor(wti_train, "Price", "Price", WINDOW_SIZE)
if len(wti_validation) > 0:
  wti_validation_inputs, wti_validation_outputs = get_input_and_output_tensor(wti_validation, "Price", "Price", WINDOW_SIZE)
wti_test_inputs, wti_test_outputs = get_input_and_output_tensor(wti_test, "Price", "Price", WINDOW_SIZE)
print("train: ", wti_train_inputs.shape, wti_train_outputs.shape)
if len(wti_validation) > 0:
  print("validation: ", wti_validation_inputs.shape, wti_validation_outputs.shape)
print("test: ", wti_test_inputs.shape, wti_test_outputs.shape)

train:  (5907, 5, 1) (5907,)
test:  (2529, 5, 1) (2529,)
time: 844 ms (started: 2024-06-11 09:49:02 +00:00)


In [ ]:
brent_train_inputs, brent_train_outputs = get_input_and_output_tensor(brent_train, "Price", "Price", WINDOW_SIZE)
if len(brent_validation) > 0:
  brent_validation_inputs, brent_validation_outputs = get_input_and_output_tensor(brent_validation, "Price", "Price", WINDOW_SIZE)
brent_test_inputs, brent_test_outputs = get_input_and_output_tensor(brent_test, "Price", "Price", WINDOW_SIZE)
print("train: ", brent_train_inputs.shape, brent_train_outputs.shape)
if len(wti_validation) > 0:
  print("validation: ", brent_validation_inputs.shape, brent_validation_outputs.shape)
print("test: ", brent_test_inputs.shape, brent_test_outputs.shape)

train:  (5840, 5, 1) (5840,)
test:  (2501, 5, 1) (2501,)
time: 1.09 s (started: 2024-06-11 09:49:03 +00:00)


### Normalize inputs

In [ ]:
wti_train_inputs = wti_train_inputs - wti_mean
wti_train_inputs = wti_train_inputs / wti_std

if len(wti_validation) > 0:
  wti_validation_inputs = wti_validation_inputs - wti_mean
  wti_validation_inputs = wti_validation_inputs / wti_std

wti_test_inputs = wti_test_inputs - wti_mean
wti_test_inputs = wti_test_inputs / wti_std

print("train: ", wti_train_inputs.shape, np.min(wti_train_inputs), np.max(wti_train_inputs))
if len(wti_validation) > 0:
  print("validation: ", wti_validation_inputs.shape, np.min(wti_validation_inputs), np.max(wti_validation_inputs))
print("test: ", wti_test_inputs.shape, np.min(wti_test_inputs), np.max(wti_test_inputs))

train:  (5907, 5, 1) -1.172327448229677 3.4520627926564575
test:  (2529, 5, 1) -0.6438355305969913 2.3545050821271585
time: 26.3 ms (started: 2024-06-11 09:49:04 +00:00)


In [ ]:
brent_train_inputs = brent_train_inputs - brent_mean
brent_train_inputs = brent_train_inputs / brent_std

if len(wti_validation) > 0:
  brent_validation_inputs = brent_validation_inputs - brent_mean
  brent_validation_inputs = brent_validation_inputs / brent_std

brent_test_inputs = brent_test_inputs - brent_mean
brent_test_inputs = brent_test_inputs / brent_std

print("train: ", brent_train_inputs.shape, np.min(brent_train_inputs), np.max(brent_train_inputs))
if len(wti_validation) > 0:
  print("validation: ", brent_validation_inputs.shape, np.min(brent_validation_inputs), np.max(brent_validation_inputs))
print("test: ", brent_test_inputs.shape, np.min(brent_test_inputs), np.max(brent_test_inputs))

train:  (5840, 5, 1) -1.1427449517660064 2.9907906274666765
test:  (2501, 5, 1) -0.6244054141699762 2.5061692147290517
time: 8.76 ms (started: 2024-06-11 09:49:04 +00:00)


## Experiments

### Compile and Fit parameters

In [ ]:
compile_parameters = {
    "optimizer": "adam",
    "loss": "mae"
}

fit_parameters = {
    "batch_size": 64,
    "epochs": 20,
    "shuffle": True,
    "verbose": 0,
}

time: 707 µs (started: 2024-06-11 09:49:04 +00:00)


### Constants

In [ ]:
NUMBER_OF_TRIALS = 10

time: 508 µs (started: 2024-06-11 09:49:04 +00:00)


### LSTM DSD Neural Network with Clustering

#### Model Name

In [ ]:
MODEL_NAME = 'Kmeans_DSD_LSTM'

time: 541 µs (started: 2024-06-11 09:49:04 +00:00)


#### Main Clustering

In [ ]:
def cluster_data():
  KmeansWTI = TimeSeriesKMeans(n_clusters=WTI_NUMBER_OF_CLUSTERS,
                        n_init=2,
                        metric="euclidean",
                        verbose=False,
                        max_iter_barycenter=10)

  KmeansWTI = KmeansWTI.fit(wti_train_inputs)

  KmeansBrent = TimeSeriesKMeans(n_clusters=BRENT_NUMBER_OF_CLUSTERS,
                        n_init=2,
                        metric="euclidean",
                        verbose=False,
                        max_iter_barycenter=10)

  KmeansBrent = KmeansBrent.fit(brent_train_inputs)

  # WTI train
  wti_train_cluster_prediction = KmeansWTI.predict(wti_train_inputs)
  wti_train_clusters_inputs = [[] for i in range(WTI_NUMBER_OF_CLUSTERS)]
  wti_train_clusters_outputs = [[] for i in range(WTI_NUMBER_OF_CLUSTERS)]

  for i in range(len(wti_train_cluster_prediction)):
    wti_train_clusters_inputs[wti_train_cluster_prediction[i]].append(wti_train_inputs[i])
    wti_train_clusters_outputs[wti_train_cluster_prediction[i]].append(wti_train_outputs[i])

  wti_train_clusters_inputs = [np.array(wti_train_clusters_inputs[i]) for i in range(WTI_NUMBER_OF_CLUSTERS)]
  wti_train_clusters_outputs = [np.array(wti_train_clusters_outputs[i]) for i in range(WTI_NUMBER_OF_CLUSTERS)]

  # WTI test
  wti_test_cluster_prediction = KmeansWTI.predict(wti_test_inputs)
  wti_test_clusters_inputs = [[] for i in range(WTI_NUMBER_OF_CLUSTERS)]
  wti_test_clusters_outputs = [[] for i in range(WTI_NUMBER_OF_CLUSTERS)]

  for i in range(len(wti_test_cluster_prediction)):
    wti_test_clusters_inputs[wti_test_cluster_prediction[i]].append(wti_test_inputs[i])
    wti_test_clusters_outputs[wti_test_cluster_prediction[i]].append(wti_test_outputs[i])

  wti_test_clusters_inputs = [np.array(wti_test_clusters_inputs[i]) for i in range(WTI_NUMBER_OF_CLUSTERS)]
  wti_test_clusters_outputs = [np.array(wti_test_clusters_outputs[i]) for i in range(WTI_NUMBER_OF_CLUSTERS)]

  # Brent train
  brent_train_cluster_prediction = KmeansBrent.predict(brent_train_inputs)
  brent_train_clusters_inputs = [[] for i in range(BRENT_NUMBER_OF_CLUSTERS)]
  brent_train_clusters_outputs = [[] for i in range(BRENT_NUMBER_OF_CLUSTERS)]

  for i in range(len(brent_train_cluster_prediction)):
    brent_train_clusters_inputs[brent_train_cluster_prediction[i]].append(brent_train_inputs[i])
    brent_train_clusters_outputs[brent_train_cluster_prediction[i]].append(brent_train_outputs[i])

  brent_train_clusters_inputs = [np.array(brent_train_clusters_inputs[i]) for i in range(BRENT_NUMBER_OF_CLUSTERS)]
  brent_train_clusters_outputs = [np.array(brent_train_clusters_outputs[i]) for i in range(BRENT_NUMBER_OF_CLUSTERS)]

  # Brent test
  brent_test_cluster_prediction = KmeansBrent.predict(brent_test_inputs)
  brent_test_clusters_inputs = [[] for i in range(BRENT_NUMBER_OF_CLUSTERS)]
  brent_test_clusters_outputs = [[] for i in range(BRENT_NUMBER_OF_CLUSTERS)]

  for i in range(len(brent_test_cluster_prediction)):
    brent_test_clusters_inputs[brent_test_cluster_prediction[i]].append(brent_test_inputs[i])
    brent_test_clusters_outputs[brent_test_cluster_prediction[i]].append(brent_test_outputs[i])

  brent_test_clusters_inputs = [np.array(brent_test_clusters_inputs[i]) for i in range(BRENT_NUMBER_OF_CLUSTERS)]
  brent_test_clusters_outputs = [np.array(brent_test_clusters_outputs[i]) for i in range(BRENT_NUMBER_OF_CLUSTERS)]

  return (wti_train_clusters_inputs,
          wti_train_clusters_outputs,
          wti_test_clusters_inputs,
          wti_test_clusters_outputs,
          brent_train_clusters_inputs,
          brent_train_clusters_outputs,
          brent_test_clusters_inputs,
          brent_test_clusters_outputs,
          wti_train_cluster_prediction,
          wti_test_cluster_prediction,
          brent_train_cluster_prediction,
          brent_test_cluster_prediction,
          KmeansWTI,
          KmeansBrent,
  )

time: 3.48 ms (started: 2024-06-11 09:49:04 +00:00)


#### DSD Classes and Functions

In [ ]:
class Sparse(Constraint):
    '''
    We will use one variable: Mask
    After we train our model dense model,
    we will save the weights and analyze them.
    We will create a mask where 1 means the
    number is far away enough from 0 and 0
    if it is to close to 0. We will multiply
    the weights by 0(making them 0) if they
    are supposed to be masked.
    '''

    def __init__(self, mask):
        self.mask = kerasBackend.cast_to_floatx(mask)

    def __call__(self,x):
        return self.mask * x

    def get_config(self):
        return {'mask': self.mask}

time: 844 µs (started: 2024-06-11 09:49:04 +00:00)


In [ ]:
def create_sparsity_masks(model,sparsity):
    weights_list = model.get_weights()
    masks = []
    for weights in weights_list:
        #We can ignore biases
        if len(weights.shape) > 1:
            weights_abs = np.abs(weights)
            masks.append((weights_abs>np.percentile(weights_abs,sparsity))*1.)
    return masks

time: 839 µs (started: 2024-06-11 09:49:04 +00:00)


#### Make the main models and fit

In [ ]:
best_wti_models = None
best_wti_error = None
best_wti_predictions = None
best_wti_cluster = None
best_brent_models = None
best_brent_error = None
best_brent_predictions = None
best_brent_cluster = None
# fig, axis = plt.subplots(NUMBER_OF_TRIALS * 2, 3)

WTI_NUMBER_OF_CLUSTERS = 8
BRENT_NUMBER_OF_CLUSTERS = 2

customized_fit_parameters = copy.deepcopy(fit_parameters)
customized_fit_parameters['epochs'] = 50

for _ in range(NUMBER_OF_TRIALS):
  print("start")

# ADDED
  if is_model_trained(MODEL_NAME + "_WTI", _) or is_model_trained(MODEL_NAME + "_BRENT", _):
    print('the model had been trained')

    brent_train_error, brent_test_error = read_model_results(MODEL_NAME + "_BRENT", _)
    KmeansBrent, brent_models = read_models_and_cluster(MODEL_NAME + "_BRENT", _)
    # wti_train_error, wti_test_error = read_model_results(MODEL_NAME + "_WTI", _)
    # KmeansWTI, wti_models = read_models_and_cluster(MODEL_NAME + "_WTI", _)

    print('brent train with clustering: ')
    show_models_results_with_labels(brent_train_outputs, brent_train_error)
    print('brent test with clustering: ')
    show_models_results_with_labels(brent_test_outputs, brent_test_error)
    # print("-"*20)
    # print('wti train with clustering: ')
    # show_models_results_with_labels(wti_train_outputs, wti_train_error)
    # print('wti test with clustering: ')
    # show_models_results_with_labels(wti_test_outputs, wti_test_error)


    # wti_test_mae = mean_absolute_error(wti_test_outputs, wti_test_error)
    brent_test_mae = mean_absolute_error(brent_test_outputs, brent_test_error)


    # if best_wti_models == None or wti_test_mae < best_wti_error:
    #   best_wti_models = wti_models
    #   best_wti_error = wti_test_mae
    #   best_wti_cluster = KmeansWTI
    #   best_wti_predictions = [wti_train_error, wti_test_error]

    if best_brent_models == None or brent_test_mae < best_brent_error:
      best_brent_models = brent_models
      best_brent_error = brent_test_mae
      best_brent_cluster = KmeansBrent
      best_brent_predictions = [brent_train_error, brent_test_error]

    print("END\n\n")
    continue


  (wti_train_clusters_inputs,
          wti_train_clusters_outputs,
          wti_test_clusters_inputs,
          wti_test_clusters_outputs,
          brent_train_clusters_inputs,
          brent_train_clusters_outputs,
          brent_test_clusters_inputs,
          brent_test_clusters_outputs,
          wti_train_cluster_prediction,
          wti_test_cluster_prediction,
          brent_train_cluster_prediction,
          brent_test_cluster_prediction,
          KmeansWTI,
          KmeansBrent,
  ) = cluster_data()
# END_ADDED

  wti_sizes = [random.randint(32, 512) for i in range(3)]
  brent_sizes = [random.randint(32, 512) for i in range(3)]

  # Dense
  # wti_model = tf.keras.Sequential([
  #     tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
  #     tf.keras.layers.LSTM(128),
  #     # tf.keras.layers.Dense(wti_sizes[0], 'leaky_relu'),
  #     # tf.keras.layers.Dense(wti_sizes[1], 'leaky_relu'),
  #     tf.keras.layers.Dense(wti_sizes[2], 'leaky_relu'),
  #     tf.keras.layers.Dense(1),
  # ])
  brent_model = tf.keras.Sequential([
      tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
      tf.keras.layers.LSTM(128),
      # tf.keras.layers.Dense(brent_sizes[0], 'leaky_relu'),
      # tf.keras.layers.Dense(brent_sizes[1], 'leaky_relu'),
      tf.keras.layers.Dense(brent_sizes[2], 'leaky_relu'),
      tf.keras.layers.Dense(1),
  ])

  # wti_model.compile(**compile_parameters)
  brent_model.compile(**compile_parameters)

  # wti_model.fit((wti_train_inputs), wti_train_outputs, **customized_fit_parameters,
  #                           validation_data=((wti_test_inputs), wti_test_outputs))
  brent_model.fit((brent_train_inputs), brent_train_outputs, **customized_fit_parameters,
                            validation_data=((brent_test_inputs), brent_test_outputs))

  # print("WTI Dense")
  # show_models_results(wti_model, wti_train_inputs, wti_train_outputs,
  #                     wti_test_inputs, wti_test_outputs, squeeze=False)
  print("Brent Dense")
  show_models_results(brent_model, brent_train_inputs, brent_train_outputs,
                      brent_test_inputs, brent_test_outputs, squeeze=False)
  # sns.histplot(flatten(wti_model.get_weights()), bins=(80), kde=True, ax=axis[_*2,0])
  # sns.histplot(flatten(brent_model.get_weights()), bins=(80), kde=True, ax=axis[_*2+1,0])

  # Sparse
  # wti_masks = create_sparsity_masks(wti_model, 40)
  brent_masks = create_sparsity_masks(brent_model, 35)

  # wti_sparse_model = tf.keras.Sequential([
  #     tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True,
  #                          kernel_constraint=(Sparse(wti_masks[0])),
  #                          recurrent_constraint=(Sparse(wti_masks[1]))),
  #     tf.keras.layers.LSTM(128,
  #                          kernel_constraint=(Sparse(wti_masks[2])),
  #                          recurrent_constraint=(Sparse(wti_masks[3]))),
  #     # tf.keras.layers.Dense(wti_sizes[0], 'leaky_relu'),
  #     # tf.keras.layers.Dense(wti_sizes[1], 'leaky_relu'),
  #     tf.keras.layers.Dense(wti_sizes[2], 'leaky_relu',
  #                           kernel_constraint=Sparse(wti_masks[4])),
  #     tf.keras.layers.Dense(1,
  #                           kernel_constraint=Sparse(wti_masks[5])),
  # ])
  brent_sparse_model = tf.keras.Sequential([
      tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True,
                           kernel_constraint=(Sparse(brent_masks[0])),
                           recurrent_constraint=(Sparse(brent_masks[1]))),
      tf.keras.layers.LSTM(128,
                           kernel_constraint=(Sparse(brent_masks[2])),
                           recurrent_constraint=(Sparse(brent_masks[3]))),
      # tf.keras.layers.Dense(brent_sizes[0], 'leaky_relu'),
      # tf.keras.layers.Dense(brent_sizes[1], 'leaky_relu'),
      tf.keras.layers.Dense(brent_sizes[2], 'leaky_relu',
                            kernel_constraint=Sparse(brent_masks[4])),
      tf.keras.layers.Dense(1,
                            kernel_constraint=Sparse(brent_masks[5])),
  ])

  # wti_sparse_model.compile(**compile_parameters)
  brent_sparse_model.compile(**compile_parameters)

  # wti_sparse_model.set_weights(wti_model.get_weights())
  brent_sparse_model.set_weights(brent_model.get_weights())

  # wti_model_history = wti_sparse_model.fit((wti_train_inputs), wti_train_outputs, **customized_fit_parameters,
  #                           validation_data=((wti_test_inputs), wti_test_outputs))
  brent_model_history = brent_sparse_model.fit((brent_train_inputs), brent_train_outputs, **customized_fit_parameters,
                            validation_data=((brent_test_inputs), brent_test_outputs))

  # sns.histplot(flatten(wti_sparse_model.get_weights()), bins=(80), kde=True, ax=axis[_*2,1])
  # sns.histplot(flatten(brent_sparse_model.get_weights()), bins=(80), kde=True, ax=axis[_*2+1,1])

  # re-Dense
  # wti_redense_model = tf.keras.Sequential([
  #     tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
  #     tf.keras.layers.LSTM(128),
  #     # tf.keras.layers.Dense(wti_sizes[0], 'leaky_relu'),
  #     # tf.keras.layers.Dense(wti_sizes[1], 'leaky_relu'),
  #     tf.keras.layers.Dense(wti_sizes[2], 'leaky_relu'),
  #     tf.keras.layers.Dense(1),
  # ])
  brent_redense_model = tf.keras.Sequential([
      tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
      tf.keras.layers.LSTM(128),
      # tf.keras.layers.Dense(brent_sizes[0], 'leaky_relu'),
      # tf.keras.layers.Dense(brent_sizes[1], 'leaky_relu'),
      tf.keras.layers.Dense(brent_sizes[2], 'leaky_relu'),
      tf.keras.layers.Dense(1),
  ])

  # wti_compile_parameters = copy.deepcopy(compile_parameters)
  # wti_compile_parameters["optimizer"] = tf.keras.optimizers.Adam(0.0001)
  brent_compile_parameters = copy.deepcopy(compile_parameters)
  brent_compile_parameters["optimizer"] = tf.keras.optimizers.Adam(0.0001)

  # wti_redense_model.compile(**wti_compile_parameters)
  brent_redense_model.compile(**brent_compile_parameters)

  # wti_redense_model.set_weights(wti_sparse_model.get_weights())
  brent_redense_model.set_weights(brent_sparse_model.get_weights())

  # wti_model_history = wti_redense_model.fit((wti_train_inputs), wti_train_outputs, **customized_fit_parameters,
  #                           validation_data=((wti_test_inputs), wti_test_outputs))
  brent_model_history = brent_redense_model.fit((brent_train_inputs), brent_train_outputs, **customized_fit_parameters,
                            validation_data=((brent_test_inputs), brent_test_outputs))

  # sns.histplot(flatten(wti_redense_model.get_weights()), bins=(80), kde=True, ax=axis[_*2,2])
  # sns.histplot(flatten(brent_redense_model.get_weights()), bins=(80), kde=True, ax=axis[_*2+1,2])

  print("-"*20)
  # print("WTI re-Dense")
  # show_models_results(wti_redense_model, wti_train_inputs, wti_train_outputs,
  #                     wti_test_inputs, wti_test_outputs, squeeze=False)
  print("Brent re-Dense")
  show_models_results(brent_redense_model, brent_train_inputs, brent_train_outputs,
                      brent_test_inputs, brent_test_outputs, squeeze=False)

  # wti_model = wti_redense_model
  brent_model = brent_redense_model

  # Make each cluster's model and fine tune it
  # wti_models = []
  brent_models = []

  # for i in range(WTI_NUMBER_OF_CLUSTERS):
  #   model = tf.keras.models.clone_model(wti_model)
  #   changed_compile_parameters = copy.deepcopy(compile_parameters)
  #   changed_compile_parameters["optimizer"] = tf.keras.optimizers.Adam(0.00001)
  #   model.compile(**changed_compile_parameters)
  #   model.set_weights(wti_model.get_weights())
  #   if len(wti_test_clusters_inputs[i]) == 0:
  #     model_history = model.fit((wti_train_clusters_inputs[i]), wti_train_clusters_outputs[i], **customized_fit_parameters)
  #   else:
  #     model_history = model.fit((wti_train_clusters_inputs[i]), wti_train_clusters_outputs[i], **customized_fit_parameters,
  #                               validation_data=((wti_test_clusters_inputs[i]), wti_test_clusters_outputs[i]))
  #   wti_models.append(model)

  for i in range(BRENT_NUMBER_OF_CLUSTERS):
    model = tf.keras.models.clone_model(brent_model)
    changed_compile_parameters = copy.deepcopy(compile_parameters)
    changed_compile_parameters["optimizer"] = tf.keras.optimizers.Adam(0.00001)
    model.compile(**changed_compile_parameters)
    model.set_weights(brent_model.get_weights())
    if len(brent_test_clusters_inputs[i]) == 0:
      model_history = model.fit((brent_train_clusters_inputs[i]), brent_train_clusters_outputs[i], **customized_fit_parameters)
    else:
      model_history = model.fit((brent_train_clusters_inputs[i]), brent_train_clusters_outputs[i], **customized_fit_parameters,
                                validation_data=((brent_test_clusters_inputs[i]), brent_test_clusters_outputs[i]))
    brent_models.append(model)

# CHANGED

  # Evaluation
  # wti_train_predicted_labels = np.zeros(wti_train_outputs.shape)
  # wti_test_predicted_labels = np.zeros(wti_test_outputs.shape)

  # for i in range(WTI_NUMBER_OF_CLUSTERS):

  #   if len(wti_test_clusters_outputs[i]) > 0:
  #     positions = np.where(wti_test_cluster_prediction == i)
  #     wti_test_predicted_labels[positions] = wti_models[i].predict((wti_test_clusters_inputs[i]), verbose=0).squeeze()

  #   if len(wti_train_clusters_outputs[i]) > 0:
  #     positions = np.where(wti_train_cluster_prediction == i)
  #     wti_train_predicted_labels[positions] = wti_models[i].predict((wti_train_clusters_inputs[i]), verbose=0).squeeze()

  brent_train_predicted_labels = np.zeros(brent_train_outputs.shape)
  brent_test_predicted_labels = np.zeros(brent_test_outputs.shape)

  for i in range(BRENT_NUMBER_OF_CLUSTERS):

    if len(brent_test_clusters_outputs[i]) > 0:
      positions = np.where(brent_test_cluster_prediction == i)
      brent_test_predicted_labels[positions] = brent_models[i].predict((brent_test_clusters_inputs[i]), verbose=0).squeeze()

    if len(brent_train_clusters_outputs[i]) > 0:
      positions = np.where(brent_train_cluster_prediction == i)
      brent_train_predicted_labels[positions] = brent_models[i].predict((brent_train_clusters_inputs[i]), verbose=0).squeeze()

  # wti_train_mae = mean_absolute_error(wti_train_outputs, wti_train_predicted_labels)
  # wti_test_mae = mean_absolute_error(wti_test_outputs, wti_test_predicted_labels)
  brent_train_mae = mean_absolute_error(brent_train_outputs, brent_train_predicted_labels)
  brent_test_mae = mean_absolute_error(brent_test_outputs, brent_test_predicted_labels)

  print("-"*20)
  # print('wti train with clustering: ')
  # show_models_results_with_labels(wti_train_outputs, wti_train_predicted_labels)
  # print('wti test with clustering: ')
  # show_models_results_with_labels(wti_test_outputs, wti_test_predicted_labels)
  print('brent train with clustering: ')
  show_models_results_with_labels(brent_train_outputs, brent_train_predicted_labels)
  print('brent test with clustering: ')
  show_models_results_with_labels(brent_test_outputs, brent_test_predicted_labels)
  print("END\n\n")
# END_CHANGED

# ADDED
  # write_models_and_cluster(wti_models, KmeansWTI, MODEL_NAME + "_WTI", _)
  # write_model_results(MODEL_NAME + "_WTI", _, wti_train_predicted_labels, wti_test_predicted_labels)
  write_models_and_cluster(brent_models, KmeansBrent, MODEL_NAME + "_BRENT", _)
  write_model_results(MODEL_NAME + "_BRENT", _, brent_train_predicted_labels, brent_test_predicted_labels)
# END_ADDED

# CHANGED

  # if best_wti_models == None or wti_test_mae < best_wti_error:
  #   best_wti_models = wti_models
  #   best_wti_error = wti_test_mae
  #   best_wti_cluster = KmeansWTI
  #   best_wti_predictions = [wti_train_error, wti_test_error]

  if best_brent_models == None or brent_test_mae < best_brent_error:
    best_brent_models = brent_models
    best_brent_error = brent_test_mae
    best_brent_cluster = KmeansBrent
    best_brent_predictions = [brent_train_predicted_labels, brent_test_predicted_labels]


# END_CHANGED
# wti_model = best_wti_model
# brent_model = best_brent_model

# ADDED
if not (is_model_trained(MODEL_NAME + "_WTI", 'best') or is_model_trained(MODEL_NAME + "_BRENT", 'best')):
  # write_models_and_cluster(best_wti_models, best_wti_cluster, MODEL_NAME + "_WTI", 'best')
  # write_model_results(MODEL_NAME + "_WTI", 'best', best_wti_predictions[0], best_wti_predictions[1])
  write_models_and_cluster(best_brent_models, best_brent_cluster, MODEL_NAME + "_BRENT", 'best')
  write_model_results(MODEL_NAME + "_BRENT", 'best', best_brent_predictions[0], best_brent_predictions[1])
# END_ADDED

start
the model had been trained
brent train with clustering: 
MAE:  0.5553376138373598
RMSE:  0.9394219713442871
MSE:  0.8825136402443866
MAPE:  0.017186639648591122
R2:  0.9984039943673709
brent test with clustering: 
MAE:  0.999119326024473
RMSE:  1.350222437139415
MSE:  1.823100629754701
MAPE:  0.013764824534243379
R2:  0.9973717562771887
END


start
the model had been trained
brent train with clustering: 
MAE:  0.5507658442732406
RMSE:  0.9362346253969566
MSE:  0.8765352737921797
MAPE:  0.016891178974504793
R2:  0.998414806105679
brent test with clustering: 
MAE:  1.0071124140370709
RMSE:  1.3590319261261774
MSE:  1.8469677762302277
MAPE:  0.01384645194552419
R2:  0.9973373485890545
END


start
the model had been trained
brent train with clustering: 
MAE:  0.553724215821044
RMSE:  0.9447409194410875
MSE:  0.8925354048663914
MAPE:  0.01702544378503854
R2:  0.9983858702364157
brent test with clustering: 
MAE:  1.0049254526704945
RMSE:  1.3588217273156336
MSE:  1.846396486625042
MAPE

#### Show Results

In [ ]:
# wti_train, wti_test = read_model_results(MODEL_NAME + "_WTI", 'best')
brent_train, brent_test = read_model_results(MODEL_NAME + "_BRENT", 'best')

# print("WTI")
# show_models_results_with_labels(wti_test_outputs, wti_test)
print("Brent")
show_models_results_with_labels(brent_test_outputs, brent_test)

Brent
MAE:  0.995953007987479
RMSE:  1.3474562333116298
MSE:  1.8156383006903651
MAPE:  0.013748337966349004
R2:  0.997382514223953
time: 582 ms (started: 2024-06-11 09:51:42 +00:00)


time: 595 ms (started: 2024-06-11 09:51:42 +00:00)
